# `pd.DataFrame.groupby()`

Este método te permite **dividir** tu `DataFrame` en "grupos" basados en los valores de **una** o **varias** categorías y luego **aplicar operaciones** de **agregación** o **transformación** a cada grupo de manera **independiente**.

In [23]:
import numpy as np
import pandas as pd

df = pd.read_csv('data/college_data_sample.csv')

Esta es la manera de llamar el método `groupby`, nos da un objeto `DataFrameGroupBy`.

In [13]:
df.groupby('Country')

Podemos iterar en un objeto `DataFrameGroupBy`, cada elemento nos da una tupla con la categoría y el ``DataFrame`` correspondiente a esa categoría.

In [12]:
countries = df.groupby('Country')

print(len(countries))

for group in countries:
    print(group)

10
('Australia',     College ID    Country  Total Students   Male  Female       CGPA  \
17   College 8  Australia           27917   2785   13350  10.000000   
28  College 37  Australia            3345   8044   -4699   8.207852   
44  College 79  Australia           28102   4869   12347   8.398590   
47  College 73  Australia           10228   8965    1263   9.707280   
60  College 11  Australia           29586  12323    6254   9.907516   
88   College 9  Australia            7509  14603   -7094   6.732975   

    Annual Family Income            Branch      Sports  \
17               3826222  Computer Science   Badminton   
28               1075704         Economics    Swimming   
44               3797180           Physics  Volleyball   
47               3594559       Mathematics    Swimming   
60               3855033           Physics  Basketball   
88                245949           Physics  Basketball   

    Research Papers Published  Placement Rate  Faculty Count  
17             

Si le pasamos más de una columna al `groupby`, cada elemento en el iterable resultante nos da lo correspondiente a cada combinación posible entre las columnas que le pasemos.

In [16]:
groups = df.groupby(['Country', 'Sports'])

print(len(groups))

for group in groups:
    print(group[0])

62
('Australia', 'Badminton')
('Australia', 'Basketball')
('Australia', 'Swimming')
('Australia', 'Volleyball')
('Canada', 'Badminton')
('Canada', 'Basketball')
('Canada', 'Football')
('Canada', 'Tennis')
('Canada', 'Volleyball')
('China', 'Athletics')
('China', 'Badminton')
('China', 'Basketball')
('China', 'Chess')
('China', 'Cricket')
('China', 'Volleyball')
('India', 'Chess')
('India', 'Football')
('India', 'Swimming')
('India', 'Tennis')
('Japan', 'Athletics')
('Japan', 'Badminton')
('Japan', 'Basketball')
('Japan', 'Chess')
('Japan', 'Cricket')
('Japan', 'Football')
('Japan', 'Tennis')
('Japan', 'Volleyball')
('Singapore', 'Athletics')
('Singapore', 'Badminton')
('Singapore', 'Chess')
('Singapore', 'Football')
('Singapore', 'Swimming')
('Singapore', 'Tennis')
('Singapore', 'Volleyball')
('South Africa', 'Athletics')
('South Africa', 'Badminton')
('South Africa', 'Basketball')
('South Africa', 'Cricket')
('South Africa', 'Football')
('South Africa', 'Tennis')
('South Africa', 'Vol

Se pueden hacer las operaciones usuales por grupo de la siguiente manera:

In [19]:
countries = df.groupby('Country')

countries['Male'].mean()

Country
Australia       8598.166667
Canada          7957.875000
China           9211.454545
India           5189.750000
Japan           6608.375000
Singapore       6986.272727
South Africa    8346.545455
Switzerland     6907.647059
UK              6842.375000
USA             6091.625000
Name: Male, dtype: float64

In [20]:
countries['Female'].mean()

Country
Australia       3570.166667
Canada           213.250000
China           2993.090909
India           7411.000000
Japan           2316.062500
Singapore       5517.454545
South Africa     904.090909
Switzerland     4425.647059
UK              4779.625000
USA             8252.125000
Name: Female, dtype: float64

In [22]:
countries['CGPA'].median()

Country
Australia       9.052935
Canada          7.832328
China           9.566726
India           9.219690
Japan           8.968480
Singapore       9.542943
South Africa    8.186094
Switzerland     8.021593
UK              8.780262
USA             8.986929
Name: CGPA, dtype: float64

De igual manera podemos aplicar el método `.agg()` si queremos aplicar una o más funciones.

In [25]:
countries['CGPA'].agg(['min', 'max'])

,min,max
Country,,
Australia,6.732975,10.000000
Canada,6.678129,9.941536
China,7.657462,10.000000
India,7.155192,10.000000
Japan,5.805870,10.000000
Singapore,7.043492,10.000000
South Africa,6.461251,10.000000
Switzerland,5.579874,10.000000
UK,6.404293,10.000000


In [28]:
groups = df.groupby(['Country', 'Branch'])

groups['CGPA'].agg(['count', 'min', 'mean', 'max'])

count        min       mean        max
Country   Branch                                                         
Australia Computer Science             1  10.000000  10.000000  10.000000
          Economics                    1   8.207852   8.207852   8.207852
          Mathematics                  1   9.707280   9.707280   9.707280
          Physics                      3   6.732975   8.346360   9.907516
Canada    Business Administration      1   7.384987   7.384987   7.384987
...                                  ...        ...        ...        ...
USA       Chemical Engineering         1   9.245600   9.245600   9.245600
          Civil Engineering            1   7.052877   7.052877   7.052877
          Computer Science             2   9.063460   9.350824   9.638189
          Mechanical Engineering       1   8.910399   8.910399   8.910399
          Physics                      1   7.937407   7.937407   7.937407

[66 rows x 4 columns]

Aplicar nuestras **propias funciones** y expresiones `lambda`.

In [31]:
countries = df.groupby('Country')

def n_more_than_mean(series):
    return len(series[series > np.mean(series)])

countries[['CGPA', 'Research Papers Published']].agg([n_more_than_mean, lambda x: len(x)])

CGPA            Research Papers Published           
             n_more_than_mean <lambda_0>          n_more_than_mean <lambda_0>
Country                                                                      
Australia                   3          6                         3          6
Canada                      3          8                         3          8
China                       8         11                         6         11
India                       2          4                         2          4
Japan                       9         16                         8         16
Singapore                   6         11                         6         11
South Africa                5         11                         7         11
Switzerland                 9         17                         8         17
UK                          4          8                         3          8
USA                         5          8                         4          8

Si queremos renombrar las columnas de la tabla resultante, podemos utilizar diccionarios.

In [34]:
countries[['CGPA', 'Research Papers Published']].agg(
    [("count", n_more_than_mean), ("len", lambda x: len(x))]
)

CGPA     Research Papers Published    
             count len                     count len
Country                                             
Australia        3   6                         3   6
Canada           3   8                         3   8
China            8  11                         6  11
India            2   4                         2   4
Japan            9  16                         8  16
Singapore        6  11                         6  11
South Africa     5  11                         7  11
Switzerland      9  17                         8  17
UK               4   8                         3   8
USA              5   8                         4   8

## **Método `.transform()`:** 

Este método nos ayuda a transformar una o varias columnas aplicando una función, así casi como `.apply()` pero aplicándolo por grupo independientemente.

* El input y output size deben ser los mismos en las funciones que le queremos aplicar.

In [39]:
def center_scale(series):
    return (series - np.mean(series)) / np.std(series, axis=0)

In [40]:
countries = df.groupby('Country')

countries[['CGPA', 'Research Papers Published']].transform(center_scale)

,CGPA,Research Papers Published
0,-0.085044,1.215920
1,0.237122,0.514959
2,0.780163,0.011854
3,-0.084565,-0.713774
4,1.062629,-1.843492
...,...,...
95,-0.132544,1.105136
96,-0.275424,1.103106
97,0.675856,1.038217
98,0.993610,0.879241


## **El método `.filter()` de un objeto `groupby`:** 

Te permite filtrar grupos enteros basándote en una función que recibe cada grupo y te devuelve `True` o `False`. Si la función devuelve `True` para un grupo, ese grupo se queda en el resultado; si devuelve `False`, se descarta.

**¿Por qué es útil?**

* **Limpieza de datos:** Ayuda a eliminar datos poco relevantes o insuficientes para tu análisis.

* **Flexibilidad:** Puedes aplicar cualquier condición compleja que se necesite, siempre y cuando la función `True` o `False`.

In [59]:
# Podemos filtrar los grupos que tengan un GCPA mayor a 8.5
def check_gcpa(dataframe):
    return dataframe['CGPA'].mean() > 8.5

groups = df.groupby(['Country', 'Branch'])

df_filtered = groups.filter(check_gcpa)

# Le quitamos todos los grupos cuyo gcpa promedio sea menor a 8.5.
len(groups), len(df_filtered.groupby(['Country', 'Branch']))

(66, 37)

In [63]:
# Podemos encontrar el GCPA promedio por Country de los países que tienen más de 5 observaciones

countries = df.groupby(['Country'])

df_filtered = countries.filter(lambda df: df['CGPA'].count() > 10)

fcountries = df_filtered.groupby('Country')

print(len(countries))
print(len(fcountries))

fcountries['CGPA'].agg(['mean', 'std'])

10
5


,mean,std
Country,,
China,9.346369,0.790980
Japan,8.770993,1.068149
Singapore,8.898307,1.162897
South Africa,8.415122,1.072729
Switzerland,7.894794,1.193534
